In [1]:
# Config
import torch
import os
# Basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# Monitoring
from tqdm.notebook import tqdm
# IO
from os.path import join, exists, basename, dirname, splitext, expanduser
from glob import glob
# Parallel processing
from joblib import Parallel, delayed
import re
from PIL import Image
import supervision as sv
from supervision.metrics import MeanAveragePrecision
from supervision.metrics import MetricTarget
from tempfile import mkdtemp

from ultralytics import YOLO


### map calculation set parameters carefully on by one


In [3]:
#mention cuda device
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
home_dir="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/"
base_state = "uttar_pradesh" ## state for which model is trained
target_state = "bihar_gen_like_uttar_pradesh" ## state for which model is to be validated
test_state = "test_bihar" ## state for which model is to be validatd during training
model_name = "yolo11m-obb.pt" ## trained model name
task="obb" ## task for which model is trained
conf=0.001 ## confidence threshold
iou=0.33 ## iou threshold
imagsz=640 ## image size
epochs=100 ## number of epochs
# ratio="same_class_count_10_120_1000_4x"

# base_dir="/home/patel_zeel/kiln_compass_24/data/"

gt_image_dir= f"../data/swinir_data/test_bihar_same_class_count_10_120_1000_4x/images"
gt_label_dir= f"../data/swinir_data/test_bihar_same_class_count_10_120_1000_4x/labels"
# gt_image_dir= "/home/patel_zeel/kiln_compass_24/data/m0_obb_without_empty_val/images"
# gt_label_dir="/home/patel_zeel/kiln_compass_24/data/m0_obb_without_empty_val/labels"
print(gt_image_dir)
print(gt_label_dir)

## class information in yml file 
data_yml_save_path=mkdtemp()
data_yml = f"""train: dummy
val: dummy
nc: 3
names: ["CFCBK", "FCBK", "Zigzag"]
"""
data_yml_path = f"{data_yml_save_path}/data.yml"
with open(data_yml_path, "w") as f:
    f.write(data_yml)

print(data_yml_path)  


class_count="10_120_1000"
type="obb"
## model path
yolo_runs_dir = f"/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/{type}/labels/"
model_path ="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb_sr/train_haryana_same_class_count_10_120_1000_4x_test_bihar_same_class_count_10_120_1000_4x_10_120_1000_4x_obb_v1_yolo11m-obb.pt_2560_8_300/weights/best.pt"
# output path
result_dir=f"{yolo_runs_dir}train_{base_state}_test_{target_state}_{class_count}_{task}_v2_{model_name}_{imagsz}_128_{epochs}_{conf}_{iou}/"
prediction_dir = join(result_dir,"labels")
print(result_dir)
print(prediction_dir)


../data/swinir_data/test_bihar_same_class_count_10_120_1000_4x/images
../data/swinir_data/test_bihar_same_class_count_10_120_1000_4x/labels
/tmp/tmpj0bsdnpx/data.yml
/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_uttar_pradesh_test_bihar_gen_like_uttar_pradesh_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_100_0.001_0.33/
/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_uttar_pradesh_test_bihar_gen_like_uttar_pradesh_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_100_0.001_0.33/labels


In [51]:
sv.DetectionDataset.from_yolo??

Signature:
sv.DetectionDataset.from_yolo(
    images_directory_path: 'str',
    annotations_directory_path: 'str',
    data_yaml_path: 'str',
    force_masks: 'bool' = False,
    is_obb: 'bool' = False,
) -> 'DetectionDataset'
Source:   
    @classmethod
    def from_yolo(
        cls,
        images_directory_path: str,
        annotations_directory_path: str,
        data_yaml_path: str,
        force_masks: bool = False,
        is_obb: bool = False,
    ) -> DetectionDataset:
        """
        Creates a Dataset instance from YOLO formatted data.

        Args:
            images_directory_path (str): The path to the
                directory containing the images.
            annotations_directory_path (str): The path to the directory
                containing the YOLO annotation files.
            data_yaml_path (str): The path to the data
                YAML file containing class information.
            force_masks (bool): If True, forces
                masks to be loaded f

In [48]:
sv.MeanAveragePrecision??

Init signature:
sv.MeanAveragePrecision(
    map50_95: 'float',
    map50: 'float',
    map75: 'float',
    per_class_ap50_95: 'np.ndarray',
) -> None
Source:        
@dataclass(frozen=True)
class MeanAveragePrecision:
    """
    Mean Average Precision for object detection tasks.

    Attributes:
        map50_95 (float): Mean Average Precision (mAP) calculated over IoU thresholds
            ranging from `0.50` to `0.95` with a step size of `0.05`.
        map50 (float): Mean Average Precision (mAP) calculated specifically at
            an IoU threshold of `0.50`.
        map75 (float): Mean Average Precision (mAP) calculated specifically at
            an IoU threshold of `0.75`.
        per_class_ap50_95 (np.ndarray): Average Precision (AP) values calculated over
            IoU thresholds ranging from `0.50` to `0.95` with a step size of `0.05`,
            provided for each individual class.
    """

    map50_95: float
    map50: float
    map75: float
    per_class_ap50_95: np

In [4]:
## Supervision dataset
sv_dataset=sv.DetectionDataset.from_yolo(gt_image_dir,gt_label_dir,data_yml_path)
print(sv_dataset)
print(len(sv_dataset))
sv_dataset.classes

## Yolo model
model = YOLO(model_path)

687


In [6]:
targets=[]
predictions=[]
for name,_,gt_detection in tqdm(sv_dataset):
    file_name=splitext(basename(name))[0]
    # prediction_path=join(prediction_dir,f"{file_name}.txt")
    ultralytics_result=model(name,imgsz=2560,iou=0.33,conf=0.001,exist_ok=True,save_txt=True,max_det=300,name=result_dir, verbose=False)[0]
    sv_detection=sv.Detections.from_ultralytics(ultralytics_result)
    targets.append(gt_detection)
    predictions.append(sv_detection)

  0%|          | 0/687 [00:00<?, ?it/s]

Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_uttar_pradesh_test_bihar_gen_like_uttar_pradesh_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_100_0.001_0.33
1366 labels saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_uttar_pradesh_test_bihar_gen_like_uttar_pradesh_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_100_0.001_0.33/labels
Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_uttar_pradesh_test_bihar_gen_like_uttar_pradesh_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_100_0.001_0.33
1366 labels saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_uttar_pradesh_test_bihar_gen_like_uttar_pradesh_10_120_1000_obb_v2_yolo11m-obb.pt_640_128_100_0.001_0.33/labels
Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/labels/train_uttar_pradesh_test_bihar_gen_like_uttar_pradesh_10_120_1000_obb_v2_yolo11

In [52]:
MeanAveragePrecision.update??

Signature:
MeanAveragePrecision.update(
    self,
    predictions: 'Union[Detections, List[Detections]]',
    targets: 'Union[Detections, List[Detections]]',
) -> 'MeanAveragePrecision'
Source:   
    def update(
        self,
        predictions: Union[Detections, List[Detections]],
        targets: Union[Detections, List[Detections]],
    ) -> MeanAveragePrecision:
        """
        Add new predictions and targets to the metric, but do not compute the result.

        Args:
            predictions (Union[Detections, List[Detections]]): The predicted detections.
            targets (Union[Detections, List[Detections]]): The ground-truth detections.

        Returns:
            (MeanAveragePrecision): The updated metric instance.
        """
        if not isinstance(predictions, list):
            predictions = [predictions]
        if not isinstance(targets, list):
            targets = [targets]

        if len(predictions) != len(targets):
            raise ValueError(
         

In [7]:
## mAP calculation 
mAP_metric = MeanAveragePrecision(class_agnostic=False)

mAP_result=mAP_metric.update(predictions,targets).compute()
class_wise_mAP=mAP_result.ap_per_class[:,0].tolist()
# print(f"mAP_result: {mAP_result}")
matched_classes=mAP_result.matched_classes.tolist()
print(f"Matched classes: {matched_classes}")
num_classes=3
final_class_wise_mAP = [0]*num_classes

for cls, mAP in zip(matched_classes, class_wise_mAP):
    print(f"cls: {cls}, mAP: {mAP}")
    final_class_wise_mAP[cls] = mAP


print(f"class_wise_mAP: {final_class_wise_mAP}")


mAP_metric = MeanAveragePrecision(class_agnostic=True)
mAP_result = mAP_metric.update(predictions, targets).compute()
class_agnostic_result = mAP_result.ap_per_class[:, 0].tolist()
# print(f"mAP_result: {mAP_result}")
print(f"class_wise_mAP: {class_wise_mAP}")
print(f"class_agnostic_result: {class_agnostic_result}")

Matched classes: [0, 1, 2]
cls: 0, mAP: 0.0
cls: 1, mAP: 0.01416628921905103
cls: 2, mAP: 0.41925698616561097
class_wise_mAP: [0.0, 0.01416628921905103, 0.41925698616561097]
class_wise_mAP: [0.0, 0.01416628921905103, 0.41925698616561097]
class_agnostic_result: [0.4211653597824483]


### Save process


In [38]:
target_state = "bihar_gen_like_haryana"
base_state = "harayana" 

In [39]:
index = pd.MultiIndex.from_tuples([], names=["Base State", "class_count", "Target State"])
result_df = pd.DataFrame(columns=["CFCBK", "FCBK", "Zigzag", "mAP", "Class-agnostic AP"], index=index)
result_df.loc[(base_state, class_count, target_state), :] = final_class_wise_mAP + [mAP_result.mAP_scores[0]] + class_agnostic_result
display(result_df)


,,,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
Base State,class_count,Target State,,,,,
harayana,10_120_1000,bihar_gen_like_haryana,0.0,0.0,0.160172,0.151809,0.151809


In [40]:
output_csv_path = f"{home_dir}result_csv/different_model_performance_cyclegan.csv"


In [41]:
# output_csv_path = f"{home_dir}result_csv/different_state_to_{target_state}_mAP_results.csv"
os.makedirs(dirname(output_csv_path), exist_ok=True)


if not os.path.exists(output_csv_path):
    result_df.to_csv(output_csv_path, header=True)
    print(f"Results saved to {output_csv_path}")
else:
    result_df.to_csv(output_csv_path, mode='a', header=False)
    print(f"Results appended to {output_csv_path}")

Results appended to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/result_csv/different_model_performance_cyclegan.csv


In [46]:
df="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/result_csv/different_model_performance_cyclegan.csv"
df = pd.read_csv(df)
df

,Base State,Target State,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
0,haryana,bihar,0.000000,0.014448,0.493067,0.497329,0.497329
1,harayana,bihar_gen_like_haryana,0.000000,0.000000,0.160172,0.151809,0.151809
2,haryana_gen_like_bihar,test_bihar,0.000000,0.002557,0.543810,0.551084,0.551084
3,uttar_pradesh,test_bihar,0.000000,0.012656,0.509686,0.524330,0.524330
4,uttar_pradesh,bihar_gen_like_uttar_pradesh,0.000000,0.000000,0.190217,0.193910,0.193910
5,uttar_pradesh_gen_like_bihar,test_bihar,0.037129,0.008624,0.537131,0.557157,0.557157


In [47]:
# display(df)
display(df.style.background_gradient(cmap='viridis',axis=0))

,Base State,Target State,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
0,haryana,bihar,0.000000,0.014448,0.493067,0.497329,0.497329
1,harayana,bihar_gen_like_haryana,0.000000,0.000000,0.160172,0.151809,0.151809
2,haryana_gen_like_bihar,test_bihar,0.000000,0.002557,0.543810,0.551084,0.551084
3,uttar_pradesh,test_bihar,0.000000,0.012656,0.509686,0.524330,0.524330
4,uttar_pradesh,bihar_gen_like_uttar_pradesh,0.000000,0.000000,0.190217,0.193910,0.193910
5,uttar_pradesh_gen_like_bihar,test_bihar,0.037129,0.008624,0.537131,0.557157,0.557157


In [5]:
base_state_2 =  ## state for which model is trained
output_csv_path_2 = f"{home_dir}result_csv/{base_state_2}_to_different_state_mAP_results.csv"

result_west_bengal=pd.read_csv(output_csv_path)
result_punjab=pd.read_csv(output_csv_path_2)
result=pd.concat([result_west_bengal,result_punjab],ignore_index=True)
display(result)

SyntaxError: invalid syntax (1003467514.py, line 1)

In [6]:
csv_path=f"../result_csv/different_state_to_bihar_same_class_count_10_120_1000_v4_mAP_results.csv"
result=pd.read_csv(csv_path)
# sort by mAP
result=result.sort_values(by='mAP',ascending=False)
display(result.style.background_gradient(cmap='viridis',axis=0))

NameError: name 'pd' is not defined

In [29]:
def apply_state_color(val):
    if val == 'West Bengal':
        return 'background-color: yellow'  # color for West Bengal (Yellow)
    elif val == 'Punjab':
        return 'background-color: green'  # color for Punjab (Green)
    else:
        return ''  # no color if it's other

# Apply the color styling for 'Base State' and 'Target State' columns
styled_result = result.style\
    .applymap(apply_state_color, subset=["Base State", "Target State"])\
    .background_gradient(subset=["CFCBK", "FCBK", "Zigzag", "mAP", "Class-agnostic AP"], cmap="coolwarm")\
    .set_properties(**{"text-align": "center"})\
    .set_table_styles([
        {"selector": "th", "props": [("font-weight", "bold"), ("text-align", "center")]},
        {"selector": "tr:nth-child(even)", "props": [("background-color", "#f9f9f9")]}
    ])

# Displaying the styled DataFrame
display(styled_result)

,Base State,Target State,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
0,west_bengal,punjab,0.000000,0.019683,0.425855,0.492828,0.492828
1,west_bengal,uttar_pradesh,0.000000,0.129238,0.197902,0.417703,0.417703
2,west_bengal,haryana,0.000000,0.004189,0.236194,0.270739,0.270739
3,west_bengal,bihar,0.000000,0.033873,0.412915,0.451084,0.451084
4,punjab,haryana,0.000000,0.008151,0.643177,0.659907,0.659907
5,punjab,bihar,0.000000,0.012281,0.338602,0.322256,0.322256
6,punjab,uttar_pradesh,0.000000,0.035343,0.137244,0.215012,0.215012
7,punjab,west_bengal,0.000000,0.016273,0.133746,0.199013,0.199013
